In [345]:
import bs4
import psycopg2
import logging
import lxml
import sys
import requests
import json
import csv
import urllib3

In [346]:
# Load data from data.json
data_file = open("data.json")
data_dict = json.load(data_file)

In [347]:
# Create get_logger() function

def get_logger():
    logger = logging.getLogger('my_logger')
    logger.setLevel(logging.DEBUG)
    logger.propagate=False
    
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(lineno)d : %(message)s')
    
    file_handler = logging.FileHandler(data_dict["file_log"])
    file_handler.setFormatter(formatter)
    file_handler.setLevel(logging.DEBUG)

    if not logger.hasHandlers():
        logger.addHandler(file_handler)
        
    return logger
    

In [348]:
# Init csv writer
csv_file = open(data_dict["file_csv"], 'w')
csv_writer = csv.writer(csv_file)

# Reset file log
with open(data_dict["file_log"], 'w'): pass

# Init file log to track history
logger = get_logger()


In [349]:
# Create connection to DBSM

def create_connection():
    f = open('password.txt', 'r') # password to my dbms
    pw = f.read()
    f.close()
    connection = psycopg2.connect(
        host = data_dict["dbms_host"],
        database = "postgres",
        user = "postgres",
        password = pw,
        port = "5432"
    )
    connection.set_session(autocommit=True)
    return connection

connection = create_connection()
logger.info(f"Connected to {connection}")

In [350]:
# Create a table in DBMS and file csv

fields = data_dict["event"]["data_field"]
table_name = data_dict["table_name"]

def create_table_dbms():
    logger.info(f"Start create table {table_name}")
    list_field_name_field_type = [f'{field["field"]} {field["type"]} ' for field in fields]
    str_field_name_field_type = ', '.join(list_field_name_field_type)
    create_table_str = f"""
        create table if not exists {table_name} (
            {str_field_name_field_type}
        )
    """
    cur = connection.cursor()
    cur.execute(create_table_str)

def create_table_csv():
    list_field_name = [f'{field["field"]}' for field in fields]
    csv_writer.writerow(list_field_name)

create_table_csv()
create_table_dbms()

In [351]:
# Crawl data from youtube and generate

total_videos = []

def crawl_data():
    # info = requests.get(data_dict["event"]["urls"])
    html_file = open("yt_html_sample_3.html")
    soup = bs4.BeautifulSoup(html_file, features = 'html5lib')
    videos = soup.find_all("ytd-rich-item-renderer", {"class" : "style-scope ytd-rich-grid-row", "items-per-row" : "3", "lockup" : "true"})
    print(len(videos))
    
    for video in videos:
        try:
            dismissible_video = video.find("div", {"id" : "dismissible"})
        except:
            continue
        
        video_id = "Unknown"
        
        try:
            video_name = dismissible_video.find("a", {"id" : "video-title-link"})["title"]
        except:
            video_name = "Unknown"
        
        try:
            video_image_link = dismissible_video.find("img", {"style" : "background-color: transparent;"})["src"]
        except:
            video_image_link = "Unknown"
            
        try:
            video_link = dismissible_video.find("a", {"id" : "thumbnail"})["href"]
        except:
            video_link = "Unknown"
            
        try:
            author_name = dismissible_video.find("a", {"id" : "avatar-link"})["title"]
        except:
            author_name = "Unknown"
            
        try:
            author_image_link = dismissible_video.find("img", {"id" : "img"})["src"]
        except:
            author_image_link = "Unknown"
        
        try:
            author_link = dismissible_video.find("a", {"id" : "avatar-link"})["href"]
        except:
            author_link = "Unknown"
        
        video_dict = {
            "video_id" : video_id,
            "video_name" : video_name,
            "video_image_link" : video_image_link,
            "video_link" : video_link,
            "author_name" : author_name,
            "author_image_link" : author_image_link,
            "author_link" : author_link
        }
        
        total_videos.append(video_dict)
    html_file.close()
    
crawl_data()
print(len(total_videos))

144
144


In [352]:
# Insert data into DB and file csv

def insert_database():
    logger.info